In [1]:
import numpy as np
import pandas as pd

In [2]:
import gpflow
import tensorflow as tf

In [3]:
import casadi as cs

In [4]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.exceptions import NotFittedError

## Load previously exported data

In [5]:
dfs_train = []
dfs_test = []

In [6]:
train_exps = ['Exp1', 'Exp3', 'Exp5', 'Exp6']
test_exps = ['Exp2', 'Exp4', 'Exp7']

In [7]:
for exp in train_exps:
    dfs_train.append(pd.read_csv(f"../Data/Good_CARNOT/{exp}_table.csv").rename(columns = {'Power': 'SimulatedHeat'}))
    
for exp in test_exps:
    dfs_test.append(pd.read_csv(f"../Data/Good_CARNOT/{exp}_table.csv").rename(columns = {'Power': 'SimulatedHeat'}))

In [8]:
#t_cols = ['time_h', 'time_m']
t_cols = []
w_cols = ['SolRad', 'OutsideTemp']
u_cols = ['SimulatedHeat']
y_cols = ['SimulatedTemp']

In [9]:
t_lags = 0
w_lags = 1
u_lags = 2
y_lags = 3

In [10]:
dict_cols = {
    't': (t_lags, t_cols),
    'w': (w_lags, w_cols),
    'u': (u_lags, u_cols),
    'y': (y_lags, y_cols)
}

Create the scaler and set up input data scaling:

In [11]:
scaler = MinMaxScaler(feature_range = (-1, 1))

In [12]:
def get_scaled_df(df, dict_cols, scaler):
    
    t_list = dict_cols['t'][1]
    w_list = dict_cols['w'][1]
    u_list = dict_cols['u'][1]
    y_list = dict_cols['y'][1]
    
    df_local = df[t_list + w_list + u_list + y_list]
    df_scaled = df_local.to_numpy()
    
    try:
        df_scaled = scaler.transform(df_scaled)
    except NotFittedError:
        df_scaled = scaler.fit_transform(df_scaled)
        
    df_scaled = pd.DataFrame(df_scaled, index = df_local.index, columns = df_local.columns)
    
    return df_scaled

In [13]:
df_train = pd.concat(dfs_train)

In [14]:
df_train = df_train[t_cols + w_cols + u_cols + y_cols]
df_train.head()

,SolRad,OutsideTemp,SimulatedHeat,SimulatedTemp
0,57.936582,22.0,-31500,23.000000
1,54.914443,22.0,-31500,20.585367
2,73.944706,22.0,-31500,20.300922
3,76.206334,22.0,-31500,20.034647
4,65.120057,22.0,-31500,19.786064


Condition number of the raw input data:

In [15]:
np.linalg.cond(df_train.to_numpy())

16199.169760599052

Fit the scaler and scale the data:

In [16]:
df_train_sc = get_scaled_df(df_train, dict_cols, scaler)

Check the condition number of the input data:

In [17]:
np.linalg.cond(df_train_sc.to_numpy())

4.946636675064373

NOTE: Condition number of scaled data is much smaller. This makes sense.

Scale the data for each experiment individually. Used for validation graphs:

In [18]:
dfs_train_sc = []
dfs_test_sc = []
for df in dfs_train:
    df_sc = get_scaled_df(df, dict_cols, scaler)
    dfs_train_sc.append(df_sc)
    
for df in dfs_test:
    df_sc = get_scaled_df(df, dict_cols, scaler)
    dfs_test_sc.append(df_sc)

Set up the function which generated the GPR input matrix from the experimental data (including all autoregressive inputs, etc.):

In [19]:
def data_to_gpr(df, dict_cols):
    
    t_list = dict_cols['t'][1]
    w_list = dict_cols['w'][1]
    u_list = dict_cols['u'][1]
    y_list = dict_cols['y'][1]
    
    df_gpr = df[t_list + w_list + u_list + y_list].copy()
    
    for lags, names in dict_cols.values():
        for name in names:
            col_idx = df_gpr.columns.get_loc(name)
            for lag in range(1, lags + 1):
                df_gpr.insert(col_idx + lag, f"{name}_{lag}", df_gpr.loc[:, name].shift(lag))

    df_gpr.dropna(inplace = True)
    
    return df_gpr

In [20]:
dfs_gpr_train = []
for df_sc in dfs_train_sc:
    dfs_gpr_train.append(data_to_gpr(df_sc, dict_cols))
df_gpr_train = pd.concat(dfs_gpr_train)
df_gpr_train.head()

,SolRad,SolRad_1,OutsideTemp,OutsideTemp_1,SimulatedHeat,SimulatedHeat_1,SimulatedHeat_2,SimulatedTemp,SimulatedTemp_1,SimulatedTemp_2,SimulatedTemp_3
3,-0.855164,-0.859463,0.058824,0.058824,-1.0,-1.0,-1.0,-0.295224,-0.270561,-0.244215,-0.020567
4,-0.876235,-0.855164,0.058824,0.058824,-1.0,-1.0,-1.0,-0.318248,-0.295224,-0.270561,-0.244215
5,-0.911207,-0.876235,0.058824,0.058824,-1.0,-1.0,-1.0,-0.340062,-0.318248,-0.295224,-0.270561
6,-0.933425,-0.911207,0.058824,0.058824,1.0,-1.0,-1.0,-0.361066,-0.340062,-0.318248,-0.295224
7,-0.952322,-0.933425,0.058824,0.058824,-1.0,1.0,-1.0,0.051533,-0.361066,-0.340062,-0.318248


In [21]:
df_gpr_train = df_gpr_train.sample(n = 500)

In [22]:
dfs_gpr_test = []
for df_sc in dfs_test_sc:
    dfs_gpr_test.append(data_to_gpr(df_sc, dict_cols))

In [23]:
df_input_train = df_gpr_train.drop(columns = dict_cols['w'][1] + dict_cols['u'][1] + dict_cols['y'][1])
df_output_train = df_gpr_train[dict_cols['y'][1]]

np_input_train = df_input_train.to_numpy()
np_output_train = df_output_train.to_numpy().reshape(-1, 1)

In [24]:
data_train = (np_input_train, np_output_train)
#pickle.dump(data_train, open(Path("data_train.pkl"), 'wb'))

In [25]:
df_input_train.head()

,SolRad_1,OutsideTemp_1,SimulatedHeat_1,SimulatedHeat_2,SimulatedTemp_1,SimulatedTemp_2,SimulatedTemp_3
176,0.320390,-0.411765,1.0,1.0,0.022707,-0.000856,-0.024018
491,-0.242351,0.058824,1.0,1.0,0.360771,0.336727,0.312594
111,-0.994599,-0.647059,1.0,-1.0,-0.674620,-0.262339,-0.282320
187,-0.515202,-0.882353,1.0,-1.0,-0.762459,-0.736413,-0.322186
335,-0.994964,-0.647059,1.0,1.0,-0.406616,-0.817147,-0.794250


In [26]:
## Define Kernel

In [27]:
nb_dims = np_input_train.shape[1]
rational_dims = np.arange(0, (dict_cols['t'][0] + 1) * len(dict_cols['t'][1]), 1)
nb_rational_dims = len(rational_dims)
squared_dims = np.arange(nb_rational_dims, nb_dims, 1)
nb_squared_dims = len(squared_dims)

In [28]:
print(f"rational: {nb_rational_dims}")
print(f"squared: {nb_squared_dims}")

rational: 0
squared: 7


In [29]:
squared_l = np.linspace(1, 1, nb_squared_dims)
rational_l = np.linspace(1, 1, nb_rational_dims)

In [30]:
variance = tf.math.reduce_variance(np_input_train)

In [303]:
k0 = gpflow.kernels.SquaredExponential(lengthscales = squared_l, active_dims = squared_dims, variance = variance)
k1 = gpflow.kernels.Constant(variance = variance)
k2 = gpflow.kernels.RationalQuadratic(lengthscales = rational_l, active_dims = rational_dims, variance = variance)
k3 = gpflow.kernels.Periodic(k2)

In [458]:
k = (k0 + k1) * k2
k = k0

## Compile Model

In [459]:
m = gpflow.models.GPR(
    data = data_train, 
    kernel = k, 
    mean_function = None,
    )

## Train Model

In [460]:
opt = gpflow.optimizers.Scipy()

In [461]:
from datetime import datetime

In [462]:
start_time = datetime.now()
opt.minimize(m.training_loss, m.trainable_variables)
print(f"Finished fitting in {datetime.now() - start_time}")

Finished fitting in 0:00:01.293315


## CasADi Optimization part

In [350]:
def get_tf_evaluator(model):
    @tf.function
    def tensor_model_eval(tf_input):
        preds = model.predict_f(tf_input)
        return preds
    return tensor_model_eval

In [364]:
tensor_model_eval = get_tf_evaluator(m)

In [399]:
m.predict_f(tf_test)

(<tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[1.03053204]])>,
 <tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[1.15445209e-06]])>)

In [400]:
tensor_model_eval(tf_test)

(<tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[1.03053322]])>,
 <tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[1.15737976e-06]])>)

In [401]:
cs_model(np_test)

DM(-7.03164e-195)

In [405]:
cs_model.update_model(m)

In [407]:
cs_model(np_test)

DM(1.03053)

In [516]:
# Package the resulting regression model in a CasADi callback
class GPR(cs.Callback):
    def __init__(self, name, model, opts={}):
        cs.Callback.__init__(self)

        print("Instance of GPR callback")
        self.model = model
        self.tf_evaluator = get_tf_evaluator(model)
        self.n_in = model.data[0].shape[1]
        self.tf_var = tf.Variable(np.ones((1, self.n_in)), dtype = tf.float64)
        # Create a variable to keep all the gradient callback references
        self.refs = []

        self.construct(name, opts)
    
    # Update tf_evaluator
    def update_model(self, model):
        self.model = model
        self.tf_evaluator = get_tf_evaluator(model)
    
    # Number of inputs/outputs
    def get_n_in(self): return 1
    def get_n_out(self): return 1
    

    # Sparsity of the input/output
    def get_sparsity_in(self,i):
        return cs.Sparsity.dense(1,self.n_in)
    def get_sparsity_out(self,i):
        return cs.Sparsity.dense(1,1)


    def eval(self, arg):
        self.tf_var.assign(arg[0])
        [mean, _] = self.tf_evaluator(self.tf_var)
        #[mean, _] = self.model.predict_f(arg[0])
        return [mean.numpy()]
    
    def has_reverse(self, nadj): return nadj==1
    def get_reverse(self, nadj, name, inames, onames, opts):
        grad_callback = GPR_grad(name, self.n_in, self.tf_evaluator, self.tf_var)
        self.refs.append(grad_callback)
        
        nominal_in = self.mx_in()
        nominal_out = self.mx_out()
        adj_seed = self.mx_out()
        return cs.Function(name, nominal_in+nominal_out+adj_seed, grad_callback.call(nominal_in), inames, onames)

In [517]:
class GPR_grad(cs.Callback):
    def __init__(self, name, n_in, tf_evaluator, tf_var, opts={}):
        cs.Callback.__init__(self)
        self.tf_evaluator = tf_evaluator
        self.n_in = n_in
        self.tf_var = tf_var


        self.construct(name, opts)

    
    def get_n_in(self): return 1
    def get_n_out(self): return 1
    
    def get_sparsity_in(self,i):
        return cs.Sparsity.dense(1,self.n_in)
    def get_sparsity_out(self,i):
        return cs.Sparsity.dense(1,self.n_in)
    

    def eval(self, arg):
        self.tf_var.assign(arg[0])
        with tf.GradientTape() as tape:
            preds = self.tf_evaluator(self.tf_var)

        grads = tape.gradient(preds, self.tf_var)
        return [grads.numpy()]

In [519]:
cs_model = GPR("gpr", m)

Instance of GPR callback


In [419]:
np_test = np.ones((1, 7))
tf_test = tf.Variable(np_test, dtype = tf.float64)

In [422]:
%%timeit
tf_test.assign(2 * np_test)
tensor_model_eval(tf_test)

5.13 ms ± 115 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [421]:
%%timeit
casual_model_eval(np_test)

13.9 ms ± 533 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [273]:
%%timeit
tf_iter = tf.Variable(np_test, dtype = tf.float64)
with tf.GradientTape() as tape:
    preds = m.predict_f(tf_iter)
grads = tape.gradient(preds, tf_iter)
grads

41.8 ms ± 381 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [117]:
%%timeit
with tf.GradientTape() as tape:
    preds = tensor_model_eval(tf_test)
grads = tape.gradient(preds, tf_test)

22.7 ms ± 733 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Multiple shooting problem formulation

In [520]:
N_horizon = 8
n_states = 7

In [521]:
X = cs.MX.sym("X", N_horizon + 1, n_states)
lbd = cs.MX.sym("lambda")
x0 = cs.MX.sym("x0", 1, n_states)
W = cs.MX.sym("W", N_horizon, 2)

In [522]:
g = []
lbg = []
ubg = []

lbx = -np.inf*np.ones(X.shape)
ubx = np.inf*np.ones(X.shape)

T_set_sc = 2.5
##
# Set up the opjective function
##

# stage cost
u_cost = cs.dot(X[:, 2], X[:, 2])

# temperature constraint
y_cost = 0.01 * cs.dot(X[:, 4], X[:, 4])

J = u_cost + y_cost

In [523]:
# Set up equality constraints for the first step
for idx in range(n_states):
    g.append(X[0, idx] - x0[0, idx])
    lbg.append(0)
    ubg.append(0)

In [524]:
# Set up equality constraints for the following steps
for idx in range(1, N_horizon + 1):
    base_col = 0
    # w
    nb_cols = dict_cols['w'][0]
    for w_idx in range(W.shape[1]):
        w_base_col = w_idx * nb_cols
        g.append(X[idx, base_col + w_base_col] - W[idx - 1, w_idx])
        lbg.append(0)
        ubg.append(0)
        for w_lag_idx in range(1, nb_cols):
            g.append(X[idx, base_col + w_base_col + w_lag_idx] - X[idx - 1, base_col + w_base_col + w_lag_idx - 1])
            lbg.append(0)
            ubg.append(0)
            
    base_col += nb_cols * W.shape[1]
    # u
    nb_cols = dict_cols['u'][0]

    lbx[idx, base_col] = -1 #lower bound on input
    ubx[idx, base_col] = 1 #upper bound on input
    for u_lag_idx in range(1, nb_cols):
        g.append(X[idx, base_col + u_lag_idx] - X[idx - 1, base_col + u_lag_idx - 1])
        lbg.append(0)
        ubg.append(0)
    
    base_col += nb_cols
    # y
    nb_cols = dict_cols['y'][0]
    g.append(X[idx, base_col] - cs_model(X[idx - 1, :]))
    lbg.append(0)
    ubg.append(0)
    for y_lag_idx in range(1, nb_cols):
        g.append(X[idx, base_col + y_lag_idx] - X[idx - 1, base_col + y_lag_idx - 1])
        lbg.append(0)
        ubg.append(0)

In [525]:
p = cs.vertcat(cs.vec(W), cs.vec(x0))

In [526]:
prob = {'f': J, 'x': cs.vec(X), 'g': cs.vertcat(*g), 'p': p}
options = {"ipopt": {"hessian_approximation": "limited-memory", "max_iter": 100,
                     "acceptable_tol": 1e-4, "tol": 1e-4,
                     "linear_solver": "ma27",
                     #"warm_start_init_point": "yes"
                     #"acceptable_obj_change_tol": 1e-5, 
                     #"mu_strategy": "adaptive",
                    }}
solver = cs.nlpsol("solver","ipopt",prob, options)

In [527]:
real_x0 = np.random.rand(*x0.shape)
real_x0

array([[0.19784954, 0.75380109, 0.20258375, 0.48384381, 0.26975897,
        0.59621339, 0.89703682]])

In [528]:
real_W = np.random.rand(*W.shape)
real_W

array([[0.870293  , 0.08461537],
       [0.65722421, 0.34585863],
       [0.70636811, 0.12671317],
       [0.78048239, 0.64362432],
       [0.50636252, 0.66469496],
       [0.77969876, 0.09749162],
       [0.25604673, 0.25789882],
       [0.1594542 , 0.46832943]])

In [529]:
real_p = cs.vertcat(cs.vec(real_W), cs.vec(real_x0))

In [530]:
cs_model(np_test)

DM(1.03058)

In [512]:
cs_model.update_model(m)

In [513]:
cs_model(np_test)

DM(1.03058)

In [536]:
res = solver(lbx = cs.vec(lbx), ubx = cs.vec(ubx), p = real_p, lbg = lbg, ubg = ubg)

This is Ipopt version 3.13.4, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:      135
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       63
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        8
                     variables with only upper bounds:        0
Total number of equality constraints.................:       55
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 8.97e-01 0.00e+00   0.0 0.00e+00    -  0.00e+00 0.00e+00   0
   1

In [445]:
## Old result

In [443]:
res['x'].reshape((N_horizon + 1, n_states))

DM(
[[0.761242, 0.36861, 0.556879, 0.104919, 0.160625, 0.774017, 0.547034], 
 [0.480629, 0.922048, -0.000639741, 0.556879, 0.2753, 0.160625, 0.774017], 
 [0.00685743, 0.247726, -0.000615418, -0.000639741, 0.187375, 0.2753, 0.160625], 
 [0.447852, 0.283534, -0.000597185, -0.000615418, 0.192971, 0.187375, 0.2753], 
 [0.624033, 0.160237, -0.000576846, -0.000597185, 0.200781, 0.192971, 0.187375], 
 [0.771545, 0.00394968, -0.000558971, -0.000576846, 0.205897, 0.200781, 0.192971], 
 [0.660621, 0.609428, -0.000568339, -0.000558971, 0.206814, 0.205897, 0.200781], 
 [0.914765, 0.539141, -0.000550041, -0.000568339, 0.221992, 0.206814, 0.205897], 
 [0.0561185, 0.209053, 0, -0.000550041, 0.236093, 0.221992, 0.206814]])

In [444]:
cs_model([0.761242, 0.36861, 0.556879, 0.104919, 0.160625, 0.774017, 0.547034])

DM(0.2753)

In [446]:
## New result

In [466]:
res['x'].reshape((N_horizon + 1, n_states))

DM(
[[0.761242, 0.36861, 0.556879, 0.104919, 0.160625, 0.774017, 0.547034], 
 [0.480629, 0.922048, -0.000640366, 0.556879, 0.275295, 0.160625, 0.774017], 
 [0.00685743, 0.247726, -0.000615877, -0.000640366, 0.187418, 0.275295, 0.160625], 
 [0.447852, 0.283534, -0.00059751, -0.000615877, 0.193019, 0.187418, 0.275295], 
 [0.624033, 0.160237, -0.000577033, -0.00059751, 0.200834, 0.193019, 0.187418], 
 [0.771545, 0.00394968, -0.000558939, -0.000577033, 0.205969, 0.200834, 0.193019], 
 [0.660621, 0.609428, -0.000568317, -0.000558939, 0.206929, 0.205969, 0.200834], 
 [0.914765, 0.539141, -0.000550168, -0.000568317, 0.22211, 0.206929, 0.205969], 
 [0.0561185, 0.209053, 0, -0.000550168, 0.236208, 0.22211, 0.206929]])

In [467]:
cs_model([0.761242, 0.36861, 0.556879, 0.104919, 0.160625, 0.774017, 0.547034])

DM(0.275295)